In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

In [2]:
import torch

print(torch.cuda.device_count())  # should print 2
print(torch.cuda.get_device_name(0))

2
NVIDIA RTX A6000


In [3]:
!pip install pandas

In [4]:
!pip install roboflow

from roboflow import Roboflow

rf = Roboflow(api_key="RUmK4wNPHx178VCDB6nH")
project = rf.workspace("academic-lsrwt").project("ev-battery-components-edfw3")
version = project.version(3)
dataset = version.download("yolov9")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Ev-Battery-Components-3 in yolov9:: 100%|██████| 4854/4854 [00:00<00:00, 11636.13it/s]


In [5]:
from ultralytics import YOLO

# choose which YOLOv8 variant to test
models = [
    "yolov8n.pt",  # nano
    "yolov8s.pt",  # small
    "yolov8m.pt",  # medium
    "yolov8l.pt",  # large
    "yolov8x.pt",  # extra-large
]

DATASET = "Ev-Battery-Components-3/data.yaml"
EPOCHS = 100
BATCH = 16
IMGSZ = 640
DEVICE = "0"  # your GPUs

for weights in models:
    name = weights.replace(".pt", "")
    print(f"\n🚀 Training {name}")
    model = YOLO(weights)
    model.train(
        data=DATASET,
        epochs=EPOCHS,
        batch=BATCH,
        imgsz=IMGSZ,
        device=DEVICE,
        project="runs/yolov8",
        name=name,
        exist_ok=True,
    )


🚀 Training yolov8n
New https://pypi.org/project/ultralytics/8.3.228 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.195 🚀 Python-3.10.18 torch-2.7.1+cu118 CUDA:0 (NVIDIA RTX A6000, 48685MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=Ev-Battery-Components-3/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8n, nbs=64, nms=

In [4]:
import numpy as np
import pandas as pd

# Combined data from YOLOv8 logs and your provided YOLOv9 metrics
# Using np.nan for missing values (e.g., Precision/Recall for v8, hardware metrics for v9)
data = [
    # Model, mAP50, mAP50-95, Params (M), GFLOPs, Inference (ms), Size (MB), Train Time (hr), Precision, Recall
    ["yolov8n", 0.599, 0.439, 2.7, 6.8, 0.6, 5.6, 0.286, np.nan, np.nan],
    ["yolov8s", 0.601, 0.447, 9.8, 23.4, 1.2, 20.0, 0.386, np.nan, np.nan],
    ["yolov8m", 0.604, 0.460, 23.2, 67.5, 2.4, 46.8, 0.634, np.nan, np.nan],
    ["yolov8l", 0.624, 0.471, 39.4, 145.2, 3.8, 79.3, 0.906, np.nan, np.nan],
    ["yolov8x", 0.611, 0.467, 61.6, 226.7, 7.7, 123.7, 1.300, np.nan, np.nan],
    ["yolov9CustomFull", 0.599, 0.443, np.nan, np.nan, np.nan, np.nan, np.nan, 0.637, 0.608],
    ["yolov9CustomHgblocks", 0.607, 0.441, np.nan, np.nan, np.nan, np.nan, np.nan, 0.719, 0.570],
    ["yolov9CustomCombine", 0.615, 0.438, np.nan, np.nan, np.nan, np.nan, np.nan, 0.693, 0.564],
]

# Define the columns for the combined table
columns = [
    "Model",
    "mAP50",
    "mAP50-95",
    "Parameters (M)",
    "GFLOPs",
    "Inference (ms)",
    "Size (MB)",
    "Train Time (hours)",
    "Precision",
    "Recall",
]

# Create pandas DataFrame
df = pd.DataFrame(data, columns=columns)

# Optional: Set 'Model' as the index for a cleaner look
# df = df.set_index("Model")

# Show the table in Jupyter
display(df)

# You can also print as Markdown (which also renders in Jupyter)
# print(df.to_markdown(index=False, floatfmt=".3f"))

,Model,mAP50,mAP50-95,Parameters (M),GFLOPs,Inference (ms),Size (MB),Train Time (hours),Precision,Recall
0,yolov8n,0.599,0.439,2.7,6.8,0.6,5.6,0.286,NaN,NaN
1,yolov8s,0.601,0.447,9.8,23.4,1.2,20.0,0.386,NaN,NaN
2,yolov8m,0.604,0.460,23.2,67.5,2.4,46.8,0.634,NaN,NaN
3,yolov8l,0.624,0.471,39.4,145.2,3.8,79.3,0.906,NaN,NaN
4,yolov8x,0.611,0.467,61.6,226.7,7.7,123.7,1.300,NaN,NaN
5,yolov9CustomFull,0.599,0.443,NaN,NaN,NaN,NaN,NaN,0.637,0.608
6,yolov9CustomHgblocks,0.607,0.441,NaN,NaN,NaN,NaN,NaN,0.719,0.570
7,yolov9CustomCombine,0.615,0.438,NaN,NaN,NaN,NaN,NaN,0.693,0.564


In [11]:
from ultralytics import YOLO

# choose which YOLOv8 variant to test
models = [
    "yolov8n.pt",  # nano
    "yolov8s.pt",  # small
    "yolov8m.pt",  # medium
    "yolov8l.pt",  # large
    "yolov8x.pt",  # extra-large
]

DATASET = "Ev-Battery-Components-3/data.yaml"
EPOCHS = 100
BATCH = 16
IMGSZ = 640
DEVICE = "0"  # your GPUs

for weights in models:
    name = weights.replace(".pt", "")
    print(f"\n🚀 Training {name}")
    model = YOLO(weights)
    model.train(
        data=DATASET,
        epochs=EPOCHS,
        batch=BATCH,
        imgsz=IMGSZ,
        device=DEVICE,
        project="runs/yolov8",
        name=name,
        exist_ok=True,
    )


🚀 Training yolov8n
New https://pypi.org/project/ultralytics/8.3.227 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.195 🚀 Python-3.10.18 torch-2.7.1+cu118 CUDA:0 (NVIDIA RTX A6000, 48685MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=tools-detection-2/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8n, nbs=64, nms=False,